# Creating a Deep Learning Forecasting model with PyTorch
In this notebook, you will create a Recurrent Neural Network (RNN) that you can use to forecast the battery cycles used for time series battery data.

In [2]:
# Install the necessary libraries directly into the notebook context

dbutils.library.installPyPI('torch')
dbutils.library.restartPython()
dbutils.library.list()

In [3]:
import uuid
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1) # ensure repeatability

import torch
import torch.nn as nn
import torch.optim as optim

pd.set_option('display.max_columns', 10)

torch.manual_seed(0) # ensure repeatability

### Download the data

The following cell will download the data set containing the daily battery time series.
The CSV file will be saved in a temporary folder on the Databricks cluster for later reuse.

In [5]:

filesToDownload = ['daily-battery-time-series-v2.csv']

for fileToDownload in filesToDownload:
  downloadCommand = 'wget -O ''/dbfs{0}/{1}'' ''https://databricksdemostore.blob.core.windows.net/data/connected-car/{1}'''.format(., fileToDownload)
  print(downloadCommand)
  ret_val = os.system(downloadCommand)
  print('System resturned value %s' % (ret_val))

A simple check to make sure our data file was correctly downloaded.

### Load the data

The previously downloaded CSV file will be loaded into a Pandas Dataframe and its first few rows inspected.

In [9]:
# Load the dataset
file_path = '/dbfs%s/%s' % (., filesToDownload[0])
df = pd.read_csv(file_path, delimiter=',')
df = df[['Date','Battery_ID','Battery_Age_Days','Number_Of_Trips','Daily_Trip_Duration','Daily_Cycles_Used', 'Lifetime_Cycles_Used', 'Battery_Rated_Cycles']]

# Inspect the data frame
df.head()

The time series related to one specific Battery_ID will be isolated and its shape checked. To keep the model simple and make is easier to understand, only one column will be used - Daily_Cycles_Used.

In case the dataset contains more time series, the process of training and prediction must be repeated for each individual series.

In [11]:
# Isolate the time series related to one Battery_ID
df_source = df[df['Battery_ID'] == 0][['Daily_Cycles_Used']]
df_source.shape

Reshape our input as a one dimensional array. This will make some of the operations we'll perform easier to follow and understand.

In [13]:
source = df_source.values.reshape(1, df_source.shape[0])[0]

### Prepare the data

Our time series prediction model will use a special kind of RNN (Recurrent Neural Network) built out of LSTM (Long Short Term Memory) cells. LSTMs are not particularly happy with very long series so we are setting the maximum limit of a time series to 250 steps (```sample_size```). Based on this value, we calculate then the maximum number of non-overlapping samples we can get from our original time series (```num_samples```).

We then consolidate these samples into two matrixes, ```input``` and ```output```. Notice they are built in a way that for every element Xn in every sample in ```input```, the corresponding element from ```output``` is equal to the one that follows Xn (which is Xn+1). The fundamental idea is that we're looking to train a model that will be capable of predicting Xn+1 based on Xn.

In [15]:
sample_size = 250
num_samples = source.shape[0] // sample_size

input = np.zeros((num_samples, sample_size))
output = np.zeros((num_samples, sample_size))

for i in range(num_samples):
  input[i] = source[-(i+1) * sample_size - 2 : -i * sample_size - 2]
  output[i] = source[-(i+1) * sample_size - 1 : -i * sample_size - 1]

Since we are using PyTorch, we're moving ```input``` and ```output``` into tensor space. The \_t notation is used to identify a variable that is a tensor.

In [17]:
input_t = torch.from_numpy(input)
target_t = torch.from_numpy(output)
print(input_t.shape)
print(target_t.shape)

### Build and train the model

Next, we define our model as a class derived from the base class ```nn.Module```. Our model contains two hidden LSTM layers of sizes ```hidden_layer1_size``` and ```hidden_layer2_size``` respectively. The output of the second LSTM layer is fed into a linear layer that will combine all components into a single output.

Each hidden layer also needs a pair of variables to hold internal state (```h_t```,```c_t``` and ```h_t2```,```c_t2``` respectively). They are used by the LSTM cells to keep track of their "memory" during the run of every epoch (implemented by the ```forward``` method). Notice how the internal state is reset at the beginning of each epoch run.

Also notice the ```future``` parameter which controls whether we want to also make predictions into the future or not. The value of this parameter will be 0 during the training process and set to a number of days when the model is called to make a prediction once it is trained.

In [19]:
# The number of nodes in the hidden layers
hidden_layer1_size = 75
hidden_layer2_size = 75

class LSTMPredictor(nn.Module):
    def __init__(self):
        super(LSTMPredictor, self).__init__()
        self.lstm1 = nn.LSTMCell(1, hidden_layer1_size)
        self.lstm2 = nn.LSTMCell(hidden_layer1_size, hidden_layer2_size)
        self.linear = nn.Linear(hidden_layer2_size, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), hidden_layer1_size, dtype=torch.double)
        c_t = torch.zeros(input.size(0), hidden_layer1_size, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), hidden_layer2_size, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), hidden_layer2_size, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

Set the number of epochs, the learning rate, the method to calculate the loss function, and the optimizer used for the backwards pass on the network during training.

In [21]:
# Increase the number of epochs for better results
epochs = 150
learning_rate = 0.5

# build the model
pred = LSTMPredictor()
pred.double() #convert all internal values to doubles
criteria = nn.MSELoss()
optimizer = torch.optim.Adam(pred.parameters(), lr=learning_rate)

Perform the actual training on the model.

For each epoch, we will perform the following steps:

- Make a prediction using the ```input_t``` input tensor
- Calculate how far is the predicted result from the expected result (stored in the ```target_t``` tensor). The distance is given by the value of the loss function, which we also save.
- Zero out the gradients (we are resetting them on each epoch)
- Trigger the backpropagation process through which we are recalibrating the internal weights of the network
- Activate the optimizer to help the recalibration process

In [23]:
losses = []

for epoch in np.arange(1, epochs + 1):
    
    print('Starting epoch %s...' % (epoch))
    
    # Feed the input through the network
    out = pred(input_t)

    # Calculate loss tensor
    loss = criteria(out, target_t)
    losses += [loss.item()]
    print('Current loss: %s' % (loss.item()))
    
    optimizer.zero_grad()
    
    # Trigger backpropagation
    loss.backward()
    # Move on
    optimizer.step()

Display the evolution of the loss function. We would expect the graph to flatline after a few initial pulses.

In [25]:
fig = plt.figure(figsize=(30,10))
plt.title('The evolution of the LOSS function during training', fontsize=10)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(np.arange(epochs), losses, 'r', linewidth=1.0)

display(fig)
plt.close()

### Predict the future

One the training process is finished, we are using the trained model to predict the values for the next 30 days. Since our sample size in ```sample_size``` we are just taking the last ```sample_size``` elements from the original time series and feed them to the model.

Notice the ```with torch.no_grad()``` option which basically tells PyTorch this is not part of any training process, hence there is no need to track the gradients on the tensors involved.

In [27]:
# The model is trained, predict the next 30 days
days_to_predict = 30

# Get the tensor with the last sample_size values
final_input = torch.from_numpy(source[-sample_size:].reshape(1, sample_size))

# No need to track gradient anymore
with torch.no_grad():
    
    y_t = pred(final_input, future=days_to_predict)
    y = y_t.detach().numpy()

The result of the prediction will contain the predicted output corresponding to the input plus a number of elements equal to the number of future days we need prediction for.

We'll just take a look at the future values predicted.

In [29]:
future_predictions = y[0, - days_to_predict:]
print(future_predictions)

Plot the last ```sample_size``` elements from the original time series in green and the predicted values for the next 30 days in red.

Please note that we are using synthetic training data and the target value was randomly generated around a mean, thus you will observe that the predictions are closer to the mean of the dataset.

In [31]:
fig = plt.figure(figsize=(30,10))
plt.title('Predict future values \n(Red values are predicted values)', fontsize=10)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(np.arange(sample_size), source[-sample_size:], 'g', linewidth=1.0)
plt.plot(np.arange(sample_size, sample_size + days_to_predict), future_predictions, 'r', linewidth=1.0)

display(fig)
plt.close()